# 1 Introduction

The section numbering in this notebook match that of the accompanying technical report. Markdown and inline comments guide technical stakeholders through executing code with a summary of rationale and outcomes provided in the report.

Execute the code in all cells in the order they appear below:

## 2.2 Workstation Preparation

### 2.2a Libraries

This notebook uses the wordcloud library for visualisation, which may not be pre-installed in your environment.  
The snippet below will install it directly into the active Jupyter kernel if necessary.

In [1]:
%pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import the all libraries required.

# Data wrangling and numeric operations
import pandas as pd
import numpy as np

# Date manipulation
import datetime as dt

# Visualisation and plot styling
import seaborn as sns
import matplotlib.pyplot as plt

# Regex for text cleaning
import re

# Word cloud generation and stopword filtering
from wordcloud import WordCloud
from wordcloud import STOPWORDS

### 2.2b Functions

In [3]:
# Data validation function for appointment data (ad, ar, nc)
# Outputs to the file to'validation_log_1.txt' to remove clutter from console
def log_data_validation(df, df_name, filename='validation_log_1.txt'):  # Write output to 'validation_log.txt', labeling each section by DataFrame name
    with open(filename, 'a') as f:                                      # Open the file in append mode to preserve previous logs
        
        f.write(f"\n=== Validation Log For {df_name} ===\n")            # Write a section header for this DataFrame's validation
        f.write("="*40 + "\n\n")                                        # Add a visual divider below the header

        f.write("Shape:\n")                                             # Write the DataFrame's shape (rows, columns)
        f.write(str(df.shape) + "\n\n")

        f.write("Non-Null Counts:\n")                                   # Write non-null counts.
        f.write(str(df.count()) + "\n\n")                               # Compare count with row count in Shape above for indications of null  

        f.write("Empty Columns:\n")                                     # Flag columns with all null values
        f.write(str(df.columns[df.isnull().all()]) + "\n\n")

        f.write("Data Types:\n")                                        # Write the metadata of the DataFrame
        f.write(str(df.dtypes) + "\n\n")

        f.write("Unique Values:\n")                                     # Count unique values in each column in DataFrame
        f.write(str(df.nunique()) + "\n\n")

        f.write("Duplicate Values:\n")                                  # Flag duplicate rows in the DataFrame
        f.write(str(df.duplicated()) + "\n\n")

        f.write("Descriptive Statistics:\n")                            # Determine the descriptive statistics of the dataset
        f.write(str(df.describe()) + "\n\n")
        
        f.write("Head:\n")                                              # Write column headers and first 5 rows
        f.write(str(df.head()) + "\n\n")

        f.write("Skewness (numeric columns):\n")                        # Measure asymmetry in the distribution of numeric columns
        f.write(str(df.skew(numeric_only=True)) + "\n\n")

        f.write("Kurtosis (numeric columns):\n")                        # Measure peakedness or tail heaviness in numeric distributions
        f.write(str(df.kurtosis(numeric_only=True)) + "\n\n")

        # outlier counts for numeric columns using IQR method
        f.write("Outlier Counts (IQR method):\n")
        outlier_counts = {
            col: len(df[(df[col] < df[col].quantile(0.25) - 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))) |
                        (df[col] > df[col].quantile(0.75) + 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25)))])
            for col in df.select_dtypes(include='number').columns
        }
        f.write(str(outlier_counts) + "\n\n")

    # Print confirmation to console when log is updated
    print(f"Detailed validation summary for '{df_name}' written to {filename}")  

    # Provide quick reference info in addition to the detail written to file
    print("\nQuick reference information below:\n")                      
    df.info()

In [4]:
# Data validation function for tweets DataFrame
# Outputs to the file to'validation_log_2.txt' to remove clutter from console
def log_tweet_validation(df, df_name='tweets', filename='validation_log_2.txt'):
    with open(filename, 'a', encoding='utf-8', errors='ignore') as f:  # ignore errors processing emoji characters
        
        f.write(f"\n=== Validation Log For {df_name} ===\n")
        f.write("="*40 + "\n\n")

        f.write("Shape:\n")
        f.write(str(df.shape) + "\n\n")

        f.write("Descriptive Statistics:\n")
        f.write(str(df.describe()) + "\n\n")

        f.write("Non-Null Counts:\n")
        f.write(str(df.count()) + "\n\n")

        f.write("Empty Columns:\n")
        f.write(str(df.columns[df.isnull().all()]) + "\n\n")

        f.write("Data Types:\n")
        f.write(str(df.dtypes) + "\n\n")

        f.write("Unique Values:\n")
        f.write(str(df.nunique()) + "\n\n")

        f.write("Duplicate Rows:\n")
        f.write(str(df.duplicated()) + "\n\n")

        f.write("Head:\n")
        f.write(str(df.head()) + "\n\n")

        f.write("Skewness (numeric columns):\n")
        f.write(str(df.skew(numeric_only=True)) + "\n\n")

        f.write("Kurtosis (numeric columns):\n")
        f.write(str(df.kurtosis(numeric_only=True)) + "\n\n")

    # Console confirmation
    print(f"Tweet validation summary written to {filename}")

In [5]:
# Assigns a season label based on numeric month, used for Seasonal Analysis
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

In [6]:
# Visualisations - small countplots
# For countplots when column count < 10
def export_countplot_short(df, column, filename, title, xlabel, ylabel, figsize=(10, 6), rotation=45):
    # Set up canvas before plotting
    fig, ax = plt.subplots(figsize=figsize)

    # Plot directly onto the axes
    sns.countplot(data=df, x=column, order=df[column].value_counts().index, ax=ax)

    # Format chart
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.tick_params(axis='x', labelrotation=90)
    plt.tight_layout()

    # Export to file in home directory and display
    fig.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

In [7]:
# Visualisations - large countplots
# For countplots when column count >= 10
def export_countplot_long(df, column, filename, title, xlabel, ylabel, figsize=(10, 6), rotation=45):
    # Set up canvas before plotting
    fig, ax = plt.subplots(figsize=figsize)

    # Plot directly onto the axes
    sns.countplot(data=df, y=column, order=df[column].value_counts().index, ax=ax)

    # Format chart
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.tick_params(axis='x', labelrotation=90)
    plt.tight_layout()

    # Export to file in home directory and display
    fig.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

In [8]:
# Visualisations - pie chart
# For pie charts with percentage labels and external legend
def export_piechart_with_legend(values, labels, filename, title, legend_title, figsize=(6, 6), startangle=90, edgecolor='white'):
    # Set up canvas before plotting
    fig, ax = plt.subplots(figsize=figsize)

    # Calculate total and format labels
    total = sum(values)
    legend_labels = [
        f"{label} ({value / total * 100:.1f}%)"
        for label, value in zip(labels, values)
    ]

    # Plot directly onto the axes
    wedges, _ = ax.pie(
        values,
        startangle=startangle,
        wedgeprops={"edgecolor": edgecolor}
    )

    # Add legend box to one side
    ax.legend(
        wedges,
        legend_labels,
        title=legend_title,
        loc='center left',
        bbox_to_anchor=(1, 0.5)
    )

    # Format chart
    ax.set_title(title, fontsize=14)
    plt.tight_layout()

    # Export to file in home directory and display
    fig.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

In [9]:
# Time series charts
# For lineplots with optional hue grouping and external legend box
def export_lineplot_with_legend(df, x, y, filename, title, xlabel, ylabel, legend_title='Legend', hue=None, figsize=(10, 6), rotation=45):
    # Set up canvas before plotting (creates a figure and axes using matplotlib)
    fig, ax = plt.subplots(figsize=figsize)

    # Plot directly onto the axes
    # Remove confidence interval
    sns.lineplot(data=df, x=x, y=y, hue=hue, errorbar=None, ax=ax)

    # Format chart
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.tick_params(axis='x', labelrotation=90)
    ax.set_ylim(bottom=0)

    # Add legend box to one side
    if hue is not None:
        ax.legend(
            title=legend_title,
            loc='center left',
            bbox_to_anchor=(1, 0.5)
        )

    plt.tight_layout()

    # Export to file in home directory and display
    fig.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

In [10]:
# Call this function for wide barplots, e.g. seasonal trend analysis
def export_barplot_with_legend(df, x, y, filename, title, xlabel, ylabel, legend_title='Legend', hue=None, figsize=(10, 6), rotation=45):
    # Set up canvas before plotting (creates a figure and axes using matplotlib)
    fig, ax = plt.subplots(figsize=figsize)

    # Plot directly onto the axes
    # Removes errorbars
    sns.barplot(data=df, x=x, y=y, hue=hue, errorbar=None, ax=ax)

    # Format chart
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.tick_params(axis='x', labelrotation=90)
    ax.set_ylim(bottom=0)

    # Move legend box to one side
    ax.legend(
        title=legend_title,
        loc='center left',
        bbox_to_anchor=(1, 0.5)
    )

    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Leave space on the right for the box

    # Export to file in home directory and display
    fig.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

# 2 Analytical Approach

Every measure used to process and manipulate the data sets to produce project deliverables. Code should be executed in the order it appears in this Notebook

### 2.3a Data Ingestion

In [11]:
# Import actual_duration.csv into a DataFrame named ad
ad = pd.read_csv('actual_duration.csv')

# Import appointments_regional.csv into a DataFrame named ar
ar = pd.read_csv('appointments_regional.csv')

# Import national_categories.xlsx into a Dataframe named nc
# Refer to section 2.1 "project materials" for notes on the file format of this data set
nc = pd.read_csv('national_categories.csv')

Tweets data is validated and manipulated separately

In [12]:
# Load the tweets data set
tweets = pd.read_csv('tweets.csv')

### 2.3b Validation Checks

Note: Each validation step (a, b & c) appends to a file named validation_log.txt in this notebook’s home directory. One section in validation_log_1.txt is created for each dataframe processed. Additionally a quick reference summary - df.info() - is printed to console.

tweets df validation summary writes to a separate file, validation_log_2.txt.

Run validation for ad dataframe imported from actual_duration.csv

In [13]:
# Validate the ad DataFrame
log_data_validation(ad, 'Actual Duration')

Detailed validation summary for 'Actual Duration' written to validation_log_1.txt

Quick reference information below:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137793 entries, 0 to 137792
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   sub_icb_location_code      137793 non-null  object
 1   sub_icb_location_ons_code  137793 non-null  object
 2   sub_icb_location_name      137793 non-null  object
 3   icb_ons_code               137793 non-null  object
 4   region_ons_code            137793 non-null  object
 5   appointment_date           137793 non-null  object
 6   actual_duration            137793 non-null  object
 7   count_of_appointments      137793 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 8.4+ MB


Run validation for ar dataframe imported from appoitnments_regional.csv

In [14]:
# Validate the ar DataFrame
log_data_validation(ar, 'Appointments Regional')

Detailed validation summary for 'Appointments Regional' written to validation_log_1.txt

Quick reference information below:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596821 entries, 0 to 596820
Data columns (total 7 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   icb_ons_code                       596821 non-null  object
 1   appointment_month                  596821 non-null  object
 2   appointment_status                 596821 non-null  object
 3   hcp_type                           596821 non-null  object
 4   appointment_mode                   596821 non-null  object
 5   time_between_book_and_appointment  596821 non-null  object
 6   count_of_appointments              596821 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 31.9+ MB


Run validation for nc dataframe imported from national_categories.csv

In [15]:
# Validate the nc DataFrame
log_data_validation(nc, 'National Categories')

Detailed validation summary for 'National Categories' written to validation_log_1.txt

Quick reference information below:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817394 entries, 0 to 817393
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Unnamed: 0             817394 non-null  int64 
 1   appointment_date       817394 non-null  object
 2   icb_ons_code           817394 non-null  object
 3   sub_icb_location_name  817394 non-null  object
 4   service_setting        817394 non-null  object
 5   context_type           817394 non-null  object
 6   national_category      817394 non-null  object
 7   count_of_appointments  817394 non-null  int64 
 8   appointment_month      817394 non-null  object
dtypes: int64(2), object(7)
memory usage: 56.1+ MB


Run validation for tweets dataframe imported from tweets.csv

In [16]:
# Validate the tweets DataFrame
log_tweet_validation(tweets, 'Tweets')

Tweet validation summary written to validation_log_2.txt


## 2.4 Data Wrangling & Exploratory Analysis

Working through the data sets to create subsets and aggregations to complete advanced validation tasks, increase familiarity with structure and contents and to build the visualisations used in this section.

### 2.4a Regional Summary

This grouping was used to verify that the datasets provided contained the full list of NHS locations at the three levels of categorisation described in the report in Figure 2: NHS Regional Structure

In [17]:
# Count unique NHS locations at each level of categorisation
geo_summary = (ad.groupby('region_ons_code').agg({'icb_ons_code': 'nunique','sub_icb_location_ons_code': 'nunique'}).reset_index())

# Append totals row
geo_summary.loc[len(geo_summary)] = ['Total',geo_summary['icb_ons_code'].sum(),geo_summary['sub_icb_location_ons_code'].sum()]

# Display without index for readability
print(geo_summary.to_string(index=False))

region_ons_code  icb_ons_code  sub_icb_location_ons_code
      E40000003             5                          5
      E40000005             6                         11
      E40000006             7                          7
      E40000007             6                         14
      E40000010             3                         27
      E40000011            11                         19
      E40000012             4                         23
          Total            42                        106


This block extracts a full breakdown of NHS locations indivudually for each Region with columns listing unique ICB codes, SICBL codes and SICBL location names. Screenshots of the contents of these DataFrames are shared in Appendix B of the report.

In [18]:
# Extract unique SICBLs and ICBs for each NHS region code
E40000003 = ad.loc[ad['region_ons_code'] == 'E40000003', ['icb_ons_code', 'sub_icb_location_ons_code', 'sub_icb_location_name']].drop_duplicates().sort_values('icb_ons_code').reset_index(drop=True)
E40000005 = ad.loc[ad['region_ons_code'] == 'E40000005', ['icb_ons_code', 'sub_icb_location_ons_code', 'sub_icb_location_name']].drop_duplicates().sort_values('icb_ons_code').reset_index(drop=True)
E40000006 = ad.loc[ad['region_ons_code'] == 'E40000006', ['icb_ons_code', 'sub_icb_location_ons_code', 'sub_icb_location_name']].drop_duplicates().sort_values('icb_ons_code').reset_index(drop=True)
E40000007 = ad.loc[ad['region_ons_code'] == 'E40000007', ['icb_ons_code', 'sub_icb_location_ons_code', 'sub_icb_location_name']].drop_duplicates().sort_values('icb_ons_code').reset_index(drop=True)
E40000010 = ad.loc[ad['region_ons_code'] == 'E40000010', ['icb_ons_code', 'sub_icb_location_ons_code', 'sub_icb_location_name']].drop_duplicates().sort_values('icb_ons_code').reset_index(drop=True)
E40000011 = ad.loc[ad['region_ons_code'] == 'E40000011', ['icb_ons_code', 'sub_icb_location_ons_code', 'sub_icb_location_name']].drop_duplicates().sort_values('icb_ons_code').reset_index(drop=True)
E40000012 = ad.loc[ad['region_ons_code'] == 'E40000012', ['icb_ons_code', 'sub_icb_location_ons_code', 'sub_icb_location_name']].drop_duplicates().sort_values('icb_ons_code').reset_index(drop=True)

### 2.4c Date Summary

Run the following code snippets on the ad and nc data sets to ensure their appointment_month columns are ISO-8601 compliant and change the dtype from object to datetime. Ignore format inference warning — fallback to dateutil is expected and safe.

In [19]:
# Change the date format of ad['appointment_date'] & nc['appointment_date']
ad['appointment_date'] = pd.to_datetime(ad['appointment_date'], errors='coerce')
nc['appointment_date'] = pd.to_datetime(nc['appointment_date'], errors='coerce')

C:\Users\toby_\AppData\Local\Temp\ipykernel_14924\2700791808.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ad['appointment_date'] = pd.to_datetime(ad['appointment_date'], errors='coerce')


Running the functions listed above will now show the dtype of the appointment_date column changed to datetime. For confirmation without repeating full data validation routines, which write to file, confirm with the following code snippets. Also check for nulls.

In [ ]:
# Confirm changed dtype of appoitnment_date in ad
ad.dtypes

In [ ]:
# Check for nulls in ad appointment_date
ad.isna().sum()

In [ ]:
# Confirm changed dtype of appoitnment_date in nc
nc.dtypes

In [ ]:
# Check for nulls in nc appointment_date
nc.isna().sum()

Run this code snippet on ar to change the dtype from object to period

In [ ]:
# Change the date format of ar['appointment_month']
ar['appointment_month'] = pd.to_datetime(ar['appointment_month']).dt.to_period('M')

Repeat the validation checks on ar to confirm change of dtype to appointment_month and check for nulls

In [ ]:
# Confirm changed dtype of appoitnment_month in ar
ar.dtypes

In [ ]:
# Check for nulls in ar appointment_month
ar.isna().sum()

Now we have dates captured in a reliable format in all three DataFrames, summarize the date range contained by each of them. The results are captured in a table in the report

In [ ]:
# Use min to get the earliest date in ad
print(ad['appointment_date'].min())

In [ ]:
# Use max to get the latest date in ad
print(ad['appointment_date'].max())

In [ ]:
# Use min to get the earliest date in nc
print(nc['appointment_date'].min())

In [ ]:
# Use max to get the latest date in nc
print(nc['appointment_date'].max())

In [ ]:
# use min to get the latest date in ar
print(ar['appointment_month'].min())

In [ ]:
# use max to get the latest date in ar
print(ar['appointment_month'].max())

### 2.4d Service Setting Summary

To count appointments, we group by the relevant category and aggregate the count_of_appointments column. While aggregation wasn’t strictly necessary at this stage, it provides a useful reference for later stages.

In [ ]:
# Group and aggregate appointment counts by service setting
# Used in pie chart in Figure 9. See "Visualisations"
ss_count = (nc.groupby('service_setting')['count_of_appointments'].sum().reset_index())

### 2.4e Appointments Monthly Summary

print() and describe() functions applied to each DataFrame created below to obtain statistics for the report. Screenshots of the outputs appear in Appendix C: Apointment Statistics.

In [ ]:
# Records per month ar
record_count = ar.groupby('appointment_month', as_index=False).size()

In [ ]:
# Appointments per month ar
appointment_summary = (ar.groupby('appointment_month')['count_of_appointments'].sum().reset_index(name='total_appointments').sort_values(by='total_appointments', ascending=False))

## 2.5 Data Wrangling Steps for Visualisations

### 2.5a Seasonal Analysis

Organise the data for the visualisations showing monthly trends using nc (section 3.1).

In [ ]:
# Create a copy of nc for monthly and seasonal analysis
nc_date = nc.copy()

In [ ]:
# Change the data type of the appointment month to string to allow for easier plotting
nc_date['appointment_month'] = nc_date['appointment_month'].astype('string')

In [ ]:
# Confirm the change was made to the data set
nc_date.dtypes

In [ ]:
# Aggregate on monthly level and determine the sum of records per month for service_setting
nc_date_ss = nc_date.groupby(['appointment_month', 'service_setting'])['count_of_appointments'].sum().reset_index()

In [ ]:
# Aggregate on monthly level and determine the sum of records per month for context_type
nc_date_ct = nc.groupby(['appointment_month', 'context_type'])['count_of_appointments'].sum().reset_index()

In [ ]:
# Aggregate on monthly level and determine the sum of records per month for context_type
nc_date_nc = nc.groupby(['appointment_month', 'national_category'])['count_of_appointments'].sum().reset_index()

Organise the data to create visualisations showing seasonal trends using nc...

In [ ]:
# Create new df to add a season column to
nc_season = nc_date.copy()

In [ ]:
# Creates a new column named 'season' in df nc_season and populates it with the corresponding season
# Calls the get_season function
nc_season['season'] = nc_season['appointment_date'].dt.month.apply(get_season)

In [ ]:
# Aggregate appointment counts by month and season to prepare seasonal trend data
nc_season_counts = (nc_season.groupby(['appointment_month', 'season'])['count_of_appointments'].sum().reset_index().sort_values('appointment_month'))

In [ ]:
# Create high level summary of seasonal totals
# Print of df in Appendix 
nc_season_totals = (nc_season_counts.groupby('season')['count_of_appointments'].sum().reset_index())

Repeat the steps applied to nc on ar, which contains a much wider date range and focuses on the General Practice Service Settng and by far the highest appointment count

In [ ]:
# Create new df to add a season column to
ar_season = ar.copy()

In [ ]:
# Creates a new column named 'season' in df ar_season and populates it with the corresponding season
# Calls the get_season function
ar_season['season'] = ar_season['appointment_month'].dt.month.apply(get_season)

In [ ]:
# Aggregate appointment counts by month and season to prepare seasonal trend data
ar_season_counts = (ar_season.groupby(['appointment_month', 'season'])['count_of_appointments'].sum().reset_index().sort_values('appointment_month'))

In [ ]:
# Create high level summary of seasonal totals
# Print of df in Appendix 
ar_season_totals = (ar_season_counts.groupby('season')['count_of_appointments'].sum().reset_index())

### 2.5b Social Media Analysis

The followng steps prepare the data for a Wordcloud used in section 3.2 and the presentation

In [ ]:
# Create a new DataFrame containing only the text.
tweets_text = tweets[['tweet_full_text']].copy()

In [ ]:
# Convert all text to strings and merge into one block
text = ''.join(tweets_text['tweet_full_text'].astype(str).tolist())

In [ ]:
# Remove punctuation and numbers
text = re.sub(r'[^A-Za-z\s]', '',text)

In [ ]:
# Convert text to lower case
text = text.lower()

In [ ]:
# Remove stopwords from text (the, and, is etc)
stopwords = set(STOPWORDS)
text = ' '.join(word for word in text.split() if word not in stopwords)

### 2.5c Local Highlights

These are the steps to prepare the data for two charts which focus on sub_icb_location_name "NHS North West London ICB - W2U3Z", section 3.3

In [ ]:
# Filter for rows matching the target sub-ICB location
nc_local = nc[nc['sub_icb_location_name'] == 'NHS North West London ICB - W2U3Z']

In [ ]:
# Group and aggregate appointment counts by service setting
# Repeat process used to create pie chart in 2.4e Appointments Monthly Summary
nc_local_count = (nc_local.groupby('service_setting')['count_of_appointments'].sum().reset_index())

In [ ]:
# Agregate on monthly level and determine the sum of records per month for service_setting
# Repeat process used to create time series chart in 2.5a Seasonal Analysis
nc_local_month = nc_local.groupby(['appointment_month', 'service_setting'])['count_of_appointments'].agg('sum').reset_index()

### 2.5d Service Utilisation & Appointment Analysis

These steps prepare data for a deep dive on utilisation and appointment trends based on the ar dataset in section 3.4 and in the presentation

In [ ]:
# Create a new DataFrame to prepare for categorical analysis of ar
ar_agg = ar.groupby(['appointment_month', 'hcp_type', 'appointment_status', 'appointment_mode', 'time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()

In [ ]:
# Change the data type of the appointment month to string to allow for easier plotting
ar_agg['appointment_month'] = ar_agg['appointment_month'].astype('string')

In [ ]:
# Create a new DataFrame where utilisation will be calculated
ar_df = ar.groupby('appointment_month')['count_of_appointments'].sum().reset_index()

In [ ]:
# Change the data type of the appointment month to string to allow for easier plotting
ar_df['appointment_month'] = ar_df['appointment_month'].astype('string')

In [ ]:
# Calculate utilisation
# Divide the sum of the monthly appointments by 30 to get a daily value
ar_df['utilisation'] = (ar_df['count_of_appointments'] / 30).round(1)

In [ ]:
# Populate a new column with a benchmark value as basis for monthly utilisation calculations
ar_df['nhs_planning_capacity'] = 1200000

In [ ]:
# Calculate the gap between actual utilisation and the NHS benchmark
ar_df['under_over_utilisation'] = ar_df['nhs_planning_capacity'] - ar_df['utilisation']

In [ ]:
ar_df.dtypes

In [ ]:
# Reshape data, stacking selected columns into 'metric' for use as hue in lineplot
plot_df = ar_df.melt(id_vars='appointment_month', value_vars=['utilisation', 'nhs_planning_capacity'], var_name='metric', value_name='value')

In [ ]:
# Dataframe with appointment status totals for reference (Appendix I in report)
status_totals = ar_agg.groupby('appointment_status')['count_of_appointments'].sum().reset_index()

In [ ]:
# Create DataFrame to analyse unattended appointments to map the time between booking and appointment date
ar_unattended = ar_agg[ar_agg['appointment_status'] == 'DNA']

In [ ]:
# Group the time_between_appointments categories and calculate total appointments for each.
ar_unattended_count = (ar_unattended.groupby('time_between_book_and_appointment')['count_of_appointments'].sum().reset_index().sort_values(by='count_of_appointments', ascending=False))

# 3 Visualisations & Insights

## 2.3 Continued Validation & Exploratory Analysis

Visualisations used in Section 2 of the report

In [ ]:
# Figure 4: Service Settings
export_countplot_short(
    df=nc,
    column='service_setting',
    filename='service_settings_distribution.png',
    title='Distribution of Service Settings',
    xlabel='Service Setting',
    ylabel='Record Count'
)

In [ ]:
# Figure 5: Context Types
export_countplot_short(
    df=nc,
    column='context_type',
    filename='context_type_distribution.png',
    title='Distribution of Context Types',
    xlabel='Context Types',
    ylabel='Record Count'
)

In [ ]:
# Figure 6: National Categories
export_countplot_long(
    df=nc,
    column='national_category',
    filename='national_categories_distribution.png',
    title='Distribution of National Categories',
    xlabel='Record Count',
    ylabel='National Category'
)

In [ ]:
# Figure 7: Appoitnment Status
export_countplot_short(
    df=ar,
    column='appointment_status',
    filename='appointment_status_distribution.png',
    title='Distribution of Appointment Statuses',
    xlabel='Appointment Status',
    ylabel='Record Count'
)

Pie chart showing number of records with Service Setting demand proportionately by appointment count in 2.3d "Service Setting Summary"

In [ ]:
# Figure 9: Service Settings
export_piechart_with_legend(
    values=ss_count['count_of_appointments'],
    labels=ss_count['service_setting'],
    filename='service_setting_pie.png',
    title='Service Setting Breakdown',
    legend_title='Service Setting'
)

## 3.1 Seasonal Analysis

Time series charts to visualise monthly trends

In [ ]:
# All appointments in the nc data set by month
# Use df nc_date with date column converted to dtype string
export_lineplot_with_legend(
    df=nc_date,
    x='appointment_month',
    y='count_of_appointments',
    hue=None,
    filename='appointments_monthly.png',
    title='Monthly Appointments',
    xlabel='Month',
    ylabel='Appointment Count',
    #legend_title='ignore'
)

In [ ]:
# Monthly trends by service setting
# Use df nc_date_ss with date column converted to dtype string
export_lineplot_with_legend(
    df=nc_date_ss,
    x='appointment_month',
    y='count_of_appointments',
    hue='service_setting',
    filename='appointments_monthly_ss.png',
    title='Monthly Appointments - Service Settings',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Service Settings'
)

In [ ]:
# Monthly trends by context type
# Use df nc_date_ct with date column converted to dtype string
export_lineplot_with_legend(
    df=nc_date_ct,
    x='appointment_month',
    y='count_of_appointments',
    hue='context_type',
    filename='appointments_monthly_ct.png',
    title='Monthly Appointments - Context Type',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Context Types'
)

In [ ]:
# Monthly trends by national category
# Use df nc_date_nc with date column converted to dtype string
export_lineplot_with_legend(
    df=nc_date_nc,
    x='appointment_month',
    y='count_of_appointments',
    hue='national_category',
    filename='appointments_monthly_nc.png',
    title='Monthly Appointments - National Category',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='National Categories'
)

Bar plot using hue to emphasise the season each month falls within

In [ ]:
# Seasonal trends for all categorisations in the nc data set
export_barplot_with_legend(
    df=nc_season_counts,
    x='appointment_month',
    y='count_of_appointments',
    hue='season',
    filename='season_trend_nc.png',
    title='Seasonal Trend - All Categories',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Season'
)

In [ ]:
# Seasonal trends for General Practice Appointments in the ar data set
# Observng a longer date range to view genuine seasonal trends over multiple years
export_barplot_with_legend(
    df=ar_season_counts,
    x='appointment_month',
    y='count_of_appointments',
    hue='season',
    filename='season_trend_ar.png',
    title='Seasonal Trend - Extended Range',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Season'
)

## 3.2 Social Media Analysis

In [ ]:
# Generate word cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    collocations=False  # prevents duplicate phrases like "health care"
).generate(text)

# Set up canvas and plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis('off')
ax.set_title("Healthcare Tweets")

# Save to same directory as notebook
fig.savefig("healthcare_wordcloud.png", dpi=300, bbox_inches='tight')

# Display in notebook
plt.show()

## 3.3 Local Highlights

In [ ]:
# Generate pie chart for service setting breakdown for NW London
export_piechart_with_legend(
    values=nc_local_count['count_of_appointments'],
    labels=nc_local_count['service_setting'],
    filename='local_service_setting_pie.png',
    title='Service Setting Breakdown - NW London',
    legend_title='Service Setting'
)

In [ ]:
# Monthly trends by service setting
export_lineplot_with_legend(
    df=nc_local_month,
    x='appointment_month',
    y='count_of_appointments',
    hue='service_setting',
    filename='local_monthly_ss.png',
    title='Monthly Appointments - NW London',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Service Settings'
)

## 3.4 Service Utilisation & Appointment Analysis

In [ ]:
# Lineplot indicating number of monthly visits
export_lineplot_with_legend(
    df=ar_df,
    x='appointment_month',
    y='count_of_appointments',
    hue=None,
    filename='monthly_appointments_ar.png',
    title='Appointments by Month',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Utilisation Key'
)

In [ ]:
# Compare monthly utilisation against NHS benchmark
export_lineplot_with_legend(
    df=plot_df,
    x='appointment_month',
    y='value',
    hue='metric',
    filename='utilisation_ar.png',
    title='Utilisation Analysis',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Utilisation Key'
)

In [ ]:
# Healthcare Professional Type trends monthly
export_lineplot_with_legend(
    df=ar_agg,
    x='appointment_month',
    y='count_of_appointments',
    hue='hcp_type',
    filename='hcp_type_ar.png',
    title='HCP Types by Month',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='HCP Types'
)

In [ ]:
# Attendance trends monthly
export_lineplot_with_legend(
    df=ar_agg,
    x='appointment_month',
    y='count_of_appointments',
    hue='appointment_status',
    filename='attendance_ar.png',
    title='Attendance Status by Month',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Appointment Status'
)

In [ ]:
# Appointment mode by month
export_lineplot_with_legend(
    df=ar_agg,
    x='appointment_month',
    y='count_of_appointments',
    hue='appointment_mode',
    filename='mode_ar.png',
    title='Appointment Mode by Month',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Appointment Mode'
)

In [ ]:
# Interval between booking and appointment date
export_lineplot_with_legend(
    df=ar_agg,
    x='appointment_month',
    y='count_of_appointments',
    hue='time_between_book_and_appointment',
    filename='time_between_ar.png',
    title='Booking Interval',
    xlabel='Month',
    ylabel='Appointment Count',
    legend_title='Interval Categories'
)

In [ ]:
# Drill down into unattended appointments to analyse interval between booking and appointment date
# Ignore the warning - there is no legend in this chart
export_barplot_with_legend(
    df=ar_unattended_count,
    x='time_between_book_and_appointment',
    y='count_of_appointments',
    hue='time_between_book_and_appointment',
    filename='unattended barplot.png',
    title='Unattended Appointments by Booking Interval',
    xlabel='Interval',
    ylabel='Appointment Count',
    legend_title=None
)